# Symbolic problem solving

Sometimes we need to compute something symbolically. In such cases you don't necessarily have to run to Matlab (or especially to chatGPT); you can easily compute fairly complex engineering tasks in Python as well.


In [ ]:
# The symbolic problem-solving package
from sympy import symbols, sin

x = symbols('x')
x * sin(x) + x**2


x**2 + x*sin(x)

Since in Python x is just a label we attach to program objects and not a symbol, the first step is to create a symbol named `x` and attach a (practically identically named) label to it.

A Symbol is an object for which operations behave differently (just like with `str` where `+` concatenates and does not add); in this case the operations are performed "symbolically".

If you don't like having to create symbolic variables with the symbols function, you can simply import them from the abc submodule:

In [ ]:
from sympy.abc import q, p
q * sin(x) + p * cos(x)

p*cos(x) + q*sin(x)

Let's see what sympy can do in calculus (of course it can do much more):


In [ ]:
# compute its derivative
from sympy import diff
diff(x * sin(x) + x**2)


x*cos(x) + 2*x + sin(x)

In [ ]:
# define a more complex custom function
from sympy import diff, ln, integrate

def f(x):
  return x * sin(x) + x**2 / ln(x)

# and compute both the derivative and integral functions:
(diff(f(x)), integrate(f(x)))


(x*cos(x) + 2*x/log(x) - x/log(x)**2 + sin(x),
 -x*cos(x) + sin(x) + Ei(3*log(x)))

In [ ]:
# we can work with functions, not just symbols!
from sympy import Function

f = Function('f')

diff( (f(x) - 3*x) / (2*f(x)) )


-(-3*x + f(x))*Derivative(f(x), x)/(2*f(x)**2) + (Derivative(f(x), x) - 3)/(2*f(x))

We can also simplify expressions or solve equations


In [ ]:
# import a few more sympy functions
from sympy import simplify, cos
x, y =  symbols('x y')
simplify( (x**2 + x)/(x*sin(y)**2 + x*cos(y)**2))


x + 1

In [ ]:
# And of course we can solve equations ...
from sympy import solve

expr =  (x + y) * (x**2 - y)
solve(expr, x) # solve the expression for x!


[-sqrt(y), sqrt(y), -y]

In [ ]:
# if we computed something we can make a regular python function from it.

symbolic = diff( x**2 / (sin(x) + x) )
symbolic


x**2*(-cos(x) - 1)/(x + sin(x))**2 + 2*x/(x + sin(x))

In [ ]:
from sympy import lambdify
# convert the symbolic formula to a regular python function:
python_function = lambdify(x, symbolic, "math")
python_function(0.23) # works with floats in the usual way
python_function(1)


0.6318578415774383

Sympy only accepts simplifications (or equalities) that are always true. There are many identities that are not always valid. For example, you may know from mathematics the identity: $(x^a)^b = x^{ab}$. However this is not true for x=-1, a=2, b=1/2! If, on the other hand, $a$ and $b$ are integers, then it is indeed true. Sympy allows us to attach assumptions to symbols, for example we can require that $a$ and $b$ are integers.


In [ ]:
a, b = symbols('a b')
simplify(2*(x**a)**b - x**(a*b))


-x**(a*b) + 2*(x**a)**b

In [ ]:
a, b = symbols('a b', integer=True)
simplify(2*(x**a)**b - x**(a*b))


x**(a*b)

Of course this is a huge package with many possibilities. Combinatorics, Logic, matrices, number theory, physical computations are all part of it. If you want to read more, you can check here:
https://docs.sympy.org/latest/index.html


## Python specifics

Sympy is a Python package, and as such it cannot (and does not want to) change Python's fundamental rules. In Python the `=` sign does not denote equality but assignment (or, more precisely, labeling), so you certainly can't write something like:
```python
(x + 1)**2 = x**2 + 2*x + 1
```
Because that would try to assign the right-hand side to the label `(x + 1)**2` (and would mostly be a syntax error).

Similarly, exponentiation is written with `**` in Python, not `^` (which is the XOR operator).

The double equals `==` already means equality, but Sympy here checks structural equality, so they are equal only if their structure is identical; it is not enough that they are algebraically equivalent.


In [ ]:
(x + 1)**2 == x**2 + 2*x + 1


False

If we want a "real" mathematical equality sign, we use the Eq constructor:


In [ ]:
from sympy import Eq
Eq(x**2 + 1, 10)


Eq(x**2 + 1, 10)

In [ ]:
solve(Eq(x**2 + 1, 10))


[-3, 3]

If we only care whether they are algebraically equivalent, the simplest way is usually to just subtract the two expressions and see if we get zero:


In [ ]:
expr1 =  (x + 1)**2
expr2 =  x**2 + 2*x + 1

# expr1 == expr2 ?
simplify(expr1-expr2)


0

In [ ]:
# or we can check with the equals method as well:
expr1.equals(expr2)


True

Under the hood Sympy converts everything to a "sympy object" and the results of operations on them are sympy objects as well.


In [ ]:
result = simplify(x - x + 1 )
result


1

In [ ]:
type(result)


sympy.core.numbers.One

In [ ]:
# be careful with division, because python's `1/3` is a floating point number
# (which is inexact) while in sympy Integer(1) / Integer(3) is an exact
# rational.

x + 1/3  # (Python evaluates 1/3 first and it becomes an 'inexact' float)


x + 0.333333333333333

In [ ]:
# in contrast:
from sympy import Integer
x + Integer(1) / Integer(3)


x + 1/3

In [ ]:
# or we can simply use rational numbers:
from sympy import Rational
x + Rational(1,3)


x + 1/3